In [ ]:
import os
import torch
import numpy as np

from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt

In [ ]:
available = torch.cuda.is_available()
curr_device = torch.cuda.current_device()
device = torch.device("cuda:0" if available else "cpu")
device_count = torch.cuda.device_count() 
device_name =  torch.cuda.get_device_name(0)


In [44]:
class RealNVP(nn.Module):
    def __init__(self, nets, nett, mask):
        super(RealNVP, self).__init__()
        
        self.mask = nn.Parameter(mask, requires_grad=False)
        self.t = torch.nn.ModuleList([nett() for _ in range(len(masks))])
        self.s = torch.nn.ModuleList([nets() for _ in range(len(masks))])
        
    def g(self, z):
        x = z
        for i in range(len(self.t)):
            x_ = x*self.mask[i]
            s = self.s[i](x_)*(1 - self.mask[i])
            t = self.t[i](x_)*(1 - self.mask[i])
            x = x_ + (1 - self.mask[i]) * (x * torch.exp(s) + t)
        return x

    def f(self, x):
        log_det_J, z = x.new_zeros(x.shape[0]), x
        for i in reversed(range(len(self.t))):
            z_ = self.mask[i] * z
            s = self.s[i](z_) * (1-self.mask[i])
            t = self.t[i](z_) * (1-self.mask[i])
            z = (1 - self.mask[i]) * (z - t) * torch.exp(-s) + z_
            log_det_J -= s.sum(dim=1)
        return z, log_det_J

In [97]:
nets = lambda: nn.Sequential(nn.Linear(10, 256), nn.LeakyReLU(), nn.Linear(256, 256), nn.LeakyReLU(), nn.Linear(256, 10), nn.Tanh())
nett = lambda: nn.Sequential(nn.Linear(10, 256), nn.LeakyReLU(), nn.Linear(256, 256), nn.LeakyReLU(), nn.Linear(256, 10))
masks = torch.from_numpy(np.array([[0], [1]] * 3).astype(np.float32))

flow = RealNVP(nets, nett, masks)

In [98]:
learning_rate = 0.001
batch_size = 1000
epochs = 10

flow = flow.to(device)
opt = torch.optim.Adam(flow.parameters(), lr=learning_rate)

In [48]:
class Dataset2D(Dataset):
    def __init__(self, data):
        mean = np.zeros(10)
        cov =  np.eye(10)
        self.size = len(data)
        self.origX = data 
        self.X = torch.tensor(self.origX).float()

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return self.X[idx, :]

In [99]:
mean = np.zeros(10)
cov = np.eye(10)
training_data = Dataset2D(np.random.multivariate_normal(mean, cov, 10))
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)

In [88]:
def my_logpdf(f, u, covar):
  for x in f:
    #print("in ")
    #print(x)
    k = 10
    y = x-u
    #print(y.shape)
    a = y.reshape(-1,1)
    #print(a.shape)
    b = t.linalg.inv(covar)
    #print(b.shape)
    d = t.matmul(y, b)
    #print(d.shape)
    e = t.matmul(d, a)
    numer = t.exp(-0.5 * e)
    f = (2 * t.pi)**k
    g = t.linalg.det(covar)
    denom = t.sqrt(f * g)
    pdf = numer / denom
  return pdf , numer

In [51]:
import torch as t

In [52]:
mu = t.zeros(10);

In [60]:
rng = np.random.RandomState(seed=10)
eigenvalues = np.linspace(1., 1000., 10)**-1
q, _ = np.linalg.qr(rng.randn(10, 10))
covariance = (q * eigenvalues**-1).dot(q.T).astype(np.float32)

covariance = t.Tensor(covariance).to(device)


In [100]:

for i in range(1):
  for x , y  in enumerate(train_dataloader):
    y = y.to(device)
    f , det = flow.f(y)
    pdf , ill= my_logpdf(f, mu.to(device), covariance)
    ill_log = t.log(ill)
    #print(y.shape)
    #print(det.shape)
    y = t.matmul(y.T , det)
    q_log = t.log(y)
    loss = (q_log - ill_log)
    loss = loss.sum()
    opt.zero_grad()
    loss.backward()
    opt.step()
  print(loss)
 

tensor(nan, device='cuda:0', grad_fn=<SumBackward0>)
